# 시퀸스 배열로 다루는 순환 신경망(RNN) 
* 문장을 학습한다는것은, 그 의미를 전달하기 위해선 각 단어가 정해진 순서대로 입력되어야 한다.
* 여러 데이터가 순서와 관계없이 입력된것 것과 다르게, 과거의 입력데이터와 나중의 입력데이터 사이의 관계를 고려해야한다.
* 이를 해결하기 위해 순환 신경망(RNN) 방법이 고안된다. 
* 여러 데이터가 순서대로 입력되었을대 잠시나마 앞서 입력된 데이터를 기억 해 놓는 방법. 
* 기억된 데이터가 얼마나 중요한지를 판단해 별도의 가중치를 줘서 다음 데이터로 넘어간다. 
* 모든 입력값에 이 작업을 순서대로 수행함으로 같은 층안에서 맴도는 성질 때문에 순환 신경망이라 한다. 

##LSTM(Long Short Term Memory)
* 한 층 안에서 반복을 주기적으로 해야되는 RNN의 특성상, 일반 신경망보다 기울기 소실 문제가 더 발생한다.
* 이문제를 해결하기 위해, 다음 층으로 기억된 값을 넘길지 안넘길지 관리하는 단계가 LSTM 이다. 
 

## 1. LSTM을 이용해 로이터 뉴스 카테고리 분류하기 
* 로이터 뉴스 데이터셋 불어오기 

In [6]:
from keras.datasets import reuters
(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words = 1000, test_split = 0.2)
#resuters 뉴스 데이터에는 토크나이저 작업을 이미 마친 데이터로, num_words 옵션은 빈도수가 1~1000번째로 높은 단어들을 불러오겠다는 옵션이다. 
# Y_ 타겟 오브젝트에는 카테고리가 인덱스로 분류된 넘버링 데이터로 들어가게된다. 

* 각 기사의 단어 수가 제각각 다름으로, 이를 동일하게 맞춰주는 전처리 함수 pad_sequence()를 사용해주어, 각 단어의 인덱스로 구성되어 있는 배열의 길이를 동일하게 맞춰준다. 

In [7]:
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(X_train, maxlen = 100)#단어수를 100개로 맞춰준다. 100개째를 넘는 단어는 버리고, 모자란 부분은 0으로 채워준다. 
x_test = pad_sequences(X_test, maxlen = 100)

#카테고리가 저장된 Y데이터를 카테고리걸 유틸을 통해 넘버링 해주자 
y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

* 데이터 전처리 과정이 끝났으니 딥러닝 구조를 빌드하자.

In [8]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

import numpy
import tensorflow as tf
import matplotlib.pyplot as plt 

seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)

model = tf.keras.Sequential()
model.add(Embedding(1000,100)) #1000 -> 불러온 단어의 총 개수, 100 -> 기사당 단어갯수(패딩한 값)
model.add(LSTM(100, activation = 'tanh')) #LSTM레이어의 활성화 함수 -> tanh, 옵션은 기사당 단어수라 볼 수 있다. 
model.add(Dense(46, activation = 'softmax'))# 다중분류로, 활성화함수는 softmax가 들어간다. 
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) #loss 에는 분류분제기 때문에 categorical_crossentropy

history  = model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 100, epochs =20)

Epoch 1/20
90/90 [==============================] - 18s 199ms/step - loss: 2.5777 - accuracy: 0.3497 - val_loss: 2.1698 - val_accuracy: 0.4488
Epoch 2/20
90/90 [==============================] - 17s 194ms/step - loss: 2.0350 - accuracy: 0.4904 - val_loss: 1.9222 - val_accuracy: 0.5116
Epoch 3/20
90/90 [==============================] - 17s 193ms/step - loss: 1.9174 - accuracy: 0.4915 - val_loss: 1.8657 - val_accuracy: 0.5129
Epoch 4/20
90/90 [==============================] - 17s 193ms/step - loss: 1.7368 - accuracy: 0.5522 - val_loss: 1.7635 - val_accuracy: 0.5499
Epoch 5/20
90/90 [==============================] - 17s 194ms/step - loss: 1.6733 - accuracy: 0.5720 - val_loss: 1.6838 - val_accuracy: 0.5744
Epoch 6/20
90/90 [==============================] - 17s 194ms/step - loss: 1.6051 - accuracy: 0.5930 - val_loss: 1.6467 - val_accuracy: 0.6002
Epoch 7/20
90/90 [==============================] - 17s 194ms/step - loss: 1.5251 - accuracy: 0.6134 - val_loss: 1.5239 - val_accuracy: 0.6269

In [9]:
print('\n Test Accuracy : %.4f' %(model.evaluate(x_test, y_test)[1]))

71/71 [==============================] - 1s 19ms/step - loss: 1.2446 - accuracy: 0.7133

 Test Accuracy : 0.7133
